In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# biasutti2019riu

> Module that implements the model from [RIU-Net: Embarrassingly simple semantic segmentation of 3D LiDAR point cloud](https://arxiv.org/abs/1905.08748).

In [ ]:
#| default_exp biasutti2019riu

In [ ]:
#| export
import torch
import torch.nn.functional as F
from torch.nn import Module, Sequential, Conv2d, BatchNorm2d, ReLU, ModuleList, ConvTranspose2d, CrossEntropyLoss
from torch.nn.init import kaiming_normal_, constant_, zeros_
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.data import DataLoader, random_split
from lightning import LightningDataModule, LightningModule
from colorcloud.behley2019iccv import SemanticKITTIDataset, UnfoldingProjection, ProjectionTransform, ProjectionToTensorTransform
from torchvision.transforms import v2
from torchmetrics.classification import Accuracy
from collections import OrderedDict
import re
import wandb
import numpy as np

In [ ]:
#| export
class Block(Sequential):
    "Convolutional block repeatedly used in the RIU-Net encoder and decoder."
    def __init__(self, in_channels, out_channels):
        super().__init__(OrderedDict([
            (f'conv1', Conv2d(in_channels, out_channels, 3, 1, 1, bias=False, padding_mode='circular')),
            (f'bn1', BatchNorm2d(out_channels, momentum=0.01)),
            (f'relu1', ReLU()),
            (f'conv2', Conv2d(out_channels, out_channels, 3, 1, 1, bias=False, padding_mode='circular')),
            (f'bn2', BatchNorm2d(out_channels, momentum=0.01)),
            (f'relu2', ReLU()),
        ]))
        self.init_params()
    
    def init_params(self):
        for n, p in self.named_parameters():
            if re.search('conv\d\.weight', n):
                kaiming_normal_(p, nonlinearity='relu')
            if re.search('bn\d\.bias', n):
                constant_(p, 0.1)

It implements the following architecture:

```{mermaid}
flowchart LR
  A(("
  Input
  (bs, in_c, h, w)")) --> B["
  Conv(3x3)
  in_c -> out_c"]
  B --> C["BatchNorm2d"]
  C --> D["ReLU"]
  D --> E["
  Conv(3x3)
  out_c -> out_c"]
  E --> F["BatchNorm2d"]
  F --> G["ReLU"]
  G --> H(("
  Output
  (bs, out_c, h, w)"))
```

Here is an example on how to use it:

In [ ]:
bs, in_c, out_c, h, w = 1, 5, 64, 64, 512
inp = torch.randn(bs, in_c, h, w)

b = Block(in_c, out_c)
outp = b(inp)
assert outp.shape == (bs, out_c, h, w)
print(outp.shape, f'== ({bs}, {out_c}, {h}, {w})')

torch.Size([1, 64, 64, 512]) == (1, 64, 64, 512)


In [ ]:
#| export
class Encoder(Module):
    "RIU-Net encoder architecture."
    def __init__(self, channels=(5, 64, 128, 256, 512, 1024)):
        super().__init__()
        self.blocks = ModuleList(
            [Block(channels[i], channels[i+1]) for i in range(len(channels)-1)]
        )
    
    def forward(self, x):
        enc_features = []
        for block in self.blocks:
            x = block(x)
            enc_features.append(x)
            x = F.max_pool2d(x, 2)
        return enc_features

It implements the following architecture:

```{mermaid}
flowchart LR
  A(("
  Input
  (bs, 5, h, w)")) --> B["
  Block
  5 -> 64"]
  B --> C["MaxPool(2x2)"]
  C --> D["
  Block
  64 -> 128"]
  D --> E["MaxPool(2x2)"]
  E --> F["
  Block
  128 -> 256"]
  F --> G["MaxPool(2x2)"]
  G --> H["
  Block
  256 -> 512"]
  H --> I["MaxPool(2x2)"]
  I --> J["
  Block
  512 -> 1024"]
  B --> L(("
  Output
  [(bs, 64, h, w),
  (bs, 128, h/2, w/2),
  (bs, 256, h/4, w/4),
  (bs, 512, h/8, w/8),
  (bs, 1024, h/16, w/16)]"))
  D --> L
  F --> L
  H --> L
  J --> L
```

Here is an example on how to use it:

In [ ]:
enc = Encoder()
outp = enc(inp)
[o.shape for o in outp]

[torch.Size([1, 64, 64, 512]),
 torch.Size([1, 128, 32, 256]),
 torch.Size([1, 256, 16, 128]),
 torch.Size([1, 512, 8, 64]),
 torch.Size([1, 1024, 4, 32])]

In [ ]:
#| export
class Decoder(Module):
    "RIU-Net decoder architecture."
    def __init__(self, channels=(1024, 512, 256, 128, 64)):
        super().__init__()
        self.upconvs = ModuleList(
            [ConvTranspose2d(channels[i], channels[i+1], 6, 2, 2, bias=False) for i in range(len(channels)-1)]
        )
        self.blocks = ModuleList(
            [Block(channels[i], channels[i+1]) for i in range(len(channels)-1)]
        )
        self.init_params()

    def init_params(self):
        for n, p in self.named_parameters():
            if re.search('upconvs\.\d\.weight', n):
                kaiming_normal_(p, nonlinearity='linear')
    
    def forward(self, enc_features):
        x = enc_features[-1]
        for i, (upconv, block) in enumerate(zip(self.upconvs, self.blocks)):
            x = upconv(x)
            x = torch.cat([x, enc_features[-(i+2)]], dim=1)
            x = block(x)
        return x

It implements the following architecture:

```{mermaid}
flowchart LR
  A(("
  Input
  [(bs, 1024, h/16, w/16),
  (bs, 512, h/8, w/8),
  (bs, 256, h/4, w/4),
  (bs, 128, h/2, w/2),
  (bs, 64, h, w)]")) --> B["
  ConvTranspose(2x2)
  1024 -> 512"]
  B --> C["
  Block
  concat(512,512) -> 512"]
  A --> C
  C --> D["
  ConvTranspose(2x2)
  512 -> 256"]
  D --> E["
  Block
  concat(256,256) -> 256"]
  A --> E
  E --> F["
  ConvTranspose(2x2)
  256 -> 128"]
  F --> G["
  Block
  concat(128,128) -> 128"]
  A --> G
  G --> H["
  ConvTranspose(2x2)
  128 -> 64"]
  H --> I["
  Block
  concat(64,64) -> 64"]
  A --> I
  I --> J(("
  Output
  (bs, 64, h, w)"))
```

Here is an example on how to use it:

In [ ]:
dec = Decoder()
fts = dec(outp)
assert fts.shape == (bs, out_c, h, w)
print(fts.shape, f'== ({bs}, {out_c}, {h}, {w})')

torch.Size([1, 64, 64, 512]) == (1, 64, 64, 512)


In [ ]:
#| export
class RIUNet(Module):
    "RIU-Net complete architecture."
    def __init__(self, in_channels=5, hidden_channels=(64, 128, 256, 512, 1024), n_classes=20):
        super().__init__()
        self.backbone = Sequential(OrderedDict([
            (f'enc', Encoder((in_channels, *hidden_channels))),
            (f'dec', Decoder(hidden_channels[::-1]))
        ]))
        self.head = Conv2d(hidden_channels[0], n_classes, 1)

    def init_params(self):
        for n, p in self.named_parameters():
            if re.search('head\.weight', n):
                kaiming_normal_(p, nonlinearity='linear')
            if re.search('head\.bias', n):
                zeros_(p)
    
    def forward(self, x):
        features = self.backbone(x)
        prediction = self.head(features)
        
        return prediction

It implements the following architecture:

```{mermaid}
flowchart LR
  A(("
  Input
  (bs, 5, h, w)")) --> B["Encoder"]
  B --> C["Decoder"]
  C --> D["
  Conv(1x1)
  64 -> 20"]
  D --> E(("
  Output
  (bs, 20, h, w)"))
```

Here is an example on how to use it:

In [ ]:
n_classes=20
model = RIUNet()
logits = model(inp)
assert logits.shape == (bs, n_classes, h, w)
print(logits.shape, f'== ({bs}, {n_classes}, {h}, {w})')

torch.Size([1, 20, 64, 512]) == (1, 20, 64, 512)


In [ ]:
#| export
class LitData(LightningDataModule):
    "Lightning DataModule to facilitate reproducibility of experiments in the original paper."
    def __init__(self, train_batch_size=8, eval_batch_size=16, num_workers=8):
        super().__init__()
        self.train_batch_size = train_batch_size
        self.eval_batch_size = eval_batch_size
        self.num_workers = num_workers
    
    def setup(self, stage: str):
        data_path = '/workspace/data'
        proj = UnfoldingProjection(W=512, H=64)
        tfms = v2.Compose([
            ProjectionTransform(proj),
            ProjectionToTensorTransform(),
        ])
        if stage == "fit":
            ds = SemanticKITTIDataset(data_path, transform=tfms)
            self.ds_train, self.ds_val = random_split(
                ds, [0.7, 0.3], generator=torch.Generator().manual_seed(42)
            )
        if stage == "test":
            self.ds_test = SemanticKITTIDataset(data_path, is_train=False, transform=tfms)
        if stage == "predict":
            self.ds_predict = SemanticKITTIDataset(data_path, is_train=False, transform=tfms)
            

    def train_dataloader(self):
        return DataLoader(self.ds_train, batch_size=self.train_batch_size, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.ds_val, batch_size=self.eval_batch_size, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.ds_test, batch_size=2*self.eval_batch_size, num_workers=self.num_workers)

    def predict_dataloader(self):
        return DataLoader(self.ds_predict, batch_size=self.eval_batch_size, num_workers=self.num_workers)

In [ ]:
#| export
class LitLearner(LightningModule):
    "Lightning Module to facilitate reproducibility of experiments in the original paper."
    def __init__(self, total_steps, debugging=False, debugging_hook=None, proj_viz_tfm=None):
        super().__init__()
        self.total_steps = total_steps
        self.input_norm = BatchNorm2d(5, affine=False, momentum=None)
        self.model = RIUNet()
        self.loss_fn = CrossEntropyLoss(reduction='none')
        self.train_accuracy = Accuracy(task="multiclass", num_classes=20)
        self.val_accuracy = Accuracy(task="multiclass", num_classes=20)
        self.debugging = debugging
        self.debugging_hook = debugging_hook
        self.proj_viz_tfm = proj_viz_tfm

        self.automatic_optimization = not debugging
        
        for n, m in self.model.named_modules():
            assert not hasattr(m, 'name')
            m.name = n

    def step_routine(self, batch, batch_idx, stage, metric):
        img, label, mask = batch
        label[~mask] = 0

        norm_img = self.input_norm(img)
        pred = self.model(norm_img)
        
        loss = self.loss_fn(pred, label)
        loss = loss[mask]
        max_loss = loss.max()
        loss = loss.mean()

        pred_f = torch.permute(pred, (0, 2, 3, 1))
        pred_f = torch.flatten(pred_f, 0, -2)
        mask_f = torch.flatten(mask)
        pred_m = pred_f[mask_f, :]
        label_m = label[mask]
        metric(pred_m, label_m)

        self.log(f"{stage}_loss_max", max_loss)
        self.log(f"{stage}_acc_step", metric)
        self.log(f"{stage}_loss", loss)

        if stage=="train" and self.debugging:
            wandb_logger = self.logger.experiment
            if self.global_step % 100 == 0:
                with torch.nn.modules.module.register_module_forward_hook(
                    self.debugging_hook(wandb_logger, self.global_step)
                ):
                    self.model(norm_img)  # run forward pass on current batch to log activations
            if self.global_step % 500 == 0:
                pred_np = pred[0].detach().cpu().numpy().argmax(0)
                label_np = label[0].detach().cpu().numpy()
                mask_np = mask[0].detach().cpu().numpy()
                _, label_img, _ = self.proj_viz_tfm(None, label_np, mask_np)
                _, pred_img, _ = self.proj_viz_tfm(None, pred_np, mask_np)
                img_cmp = np.concatenate((pred_img, label_img), axis=0)
                img_cmp = wandb.Image(img_cmp)
                wandb_logger.log({"examples": img_cmp}, step=self.global_step)
        
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.step_routine(batch, batch_idx, "train", self.train_accuracy)
        if not self.debugging:
            return loss
        optimizer = self.optimizers()
        optimizer.zero_grad()
        self.manual_backward(loss)
        wandb_logger = self.logger.experiment
        step = self.global_step
        p_old = {}
        for n, p in self.model.named_parameters():
            p_old[n] = p.detach().clone()
        optimizer.step()
        for n, p in self.model.named_parameters():
            optim_step = p.detach() - p_old[n]
            wandb_logger.log({"ud/" + str(n): (optim_step.std()/(p_old[n].std() + 1e-5)).log10()}, step=step)
        lr_scheduler = self.lr_schedulers()
        lr_scheduler.step()

    def on_train_epoch_end(self):
        self.log('train_acc_epoch', self.train_accuracy)

    def validation_step(self, batch, batch_idx):
        self.step_routine(batch, batch_idx, "val", self.val_accuracy)

    def on_validation_epoch_end(self):
        self.log('val_acc_epoch', self.val_accuracy)

    def configure_optimizers(self):
        lr = 1e-3
        optimizer = Adam(self.model.parameters(), lr=lr, eps=1e-5)
        lr_scheduler = OneCycleLR(optimizer, lr, self.total_steps)
        return {'optimizer': optimizer, 'lr_scheduler': lr_scheduler}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()